Wazzup

A small VQE for the ground state, from past IBM challenges, using sampler and estimators (of qiskit). 

In [1]:
# define Hamiltonian
from qiskit.opflow import PauliSumOp
H = PauliSumOp.from_list([('XYII', 1), ('IYZI', 2), ('IIZX', 3), ('XIII', 4), ('IYII', 5)])

print(H)

1.0 * XYII
+ 2.0 * IYZI
+ 3.0 * IIZX
+ 4.0 * XIII
+ 5.0 * IYII


In [2]:
#define ansatz with 16 angle variables (16 = 2^4)
from qiskit.circuit.library import EfficientSU2

qc = EfficientSU2(num_qubits=H.num_qubits, reps=1)

qc_with_meas = qc.measure_all(inplace=False)
qc_with_meas.decompose().draw(fold=120)

┌──────────┐┌──────────┐                                ┌──────────┐┌───────────┐ ░ ┌─┐         
   q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├────────────────────────■───────┤ Ry(θ[8]) ├┤ Rz(θ[12]) ├─░─┤M├─────────
        ├──────────┤├──────────┤                      ┌─┴─┐     ├──────────┤├───────────┤ ░ └╥┘┌─┐      
   q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├───────────■──────────┤ X ├─────┤ Ry(θ[9]) ├┤ Rz(θ[13]) ├─░──╫─┤M├──────
        ├──────────┤├──────────┤         ┌─┴─┐    ┌───┴───┴───┐┌┴──────────┤└───────────┘ ░  ║ └╥┘┌─┐   
   q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├──■──────┤ X ├────┤ Ry(θ[10]) ├┤ Rz(θ[14]) ├──────────────░──╫──╫─┤M├───
        ├──────────┤├──────────┤┌─┴─┐┌───┴───┴───┐├───────────┤└───────────┘              ░  ║  ║ └╥┘┌─┐
   q_3: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├┤ X ├┤ Ry(θ[11]) ├┤ Rz(θ[15]) ├───────────────────────────░──╫──╫──╫─┤M├
        └──────────┘└──────────┘└───┘└───────────┘└───────────┘                           ░  ║  ║  ║ └╥┘
meas: 4/═════════════════════════════════════════════════════════════════════════════════════╩══╩══╩══╩═
                                                                                             0  1  2  3

In [3]:
#sample the circuit:

# define a set of (random) parameter values for the ansatz circuit
import numpy as np
theta = np.random.rand(qc.num_parameters)

# use the Sampler to sample from this circuit
from qiskit.primitives import Sampler

sampler = Sampler([qc_with_meas])
s_result = sampler.run([qc_with_meas], [theta]).result()
print(s_result.quasi_dists[0])
#this gives you a list of the 16 possible bitstring outputs and their probabilities

{0: 0.3199035315788026, 1: 0.03354139975046105, 2: 0.011865238829641751, 3: 0.06978849958654942, 4: 0.027869794446542983, 5: 0.0041926446695048775, 6: 0.0050992675021793565, 7: 0.012254742706031382, 8: 0.24470677794206513, 9: 0.026397475993841006, 10: 0.010745875551538712, 11: 0.05686283794284751, 12: 0.11124408285147971, 13: 0.014599772948791393, 14: 0.012490839403456345, 15: 0.03843721829626631}


In [4]:
#find the energy for this distribution of states (i.e. for this random angle theta)
from qiskit.primitives import Estimator

estimator = Estimator([qc], [H])
e_result = estimator.run([qc], [H], [theta]).result()
print(e_result)

EstimatorResult(values=array([5.51844932]), metadata=[{}])


Exact ground state is -13. Can we get close to that by classically optimizing the $\theta$ parameters?

In [12]:
# define objective as expectation value of Hamiltonian with circuit ansatz

objective = lambda x: estimator.run([qc], [H], [x]).result().values[0]
# instantiate optimizer

from qiskit.algorithms.optimizers import SPSA
optimizer = SPSA(maxiter=500)

#Now, we can pass our objective function to the classical optimizer to find an approximate solution.

# define initial values for our circuit parameters
x0 = np.random.rand(qc.num_parameters)

# minimize the objective function
result = optimizer.minimize(objective, x0=x0)

# store ground state parameters for later
ground_state_params = result.x

# print the resulting ground state energy
print(result.fun)

-12.999907785626158


There you go, we get very close to it!